In [1]:
import pandas as pd
import numpy as np
import re
import math
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
## Loading Data
# Main tables
new=pd.read_excel('Produkty.xlsx')
old=pd.read_excel('Artikly.xlsx')

# Scraped data
mcat=pd.read_csv('dataset_moda-category.csv')
mprod=pd.read_csv('dataset_moda-product.csv')
mdet=pd.read_csv('dataset_moda-detail.csv')
scat=pd.read_csv('dataset_sport-category.csv')
sprod=pd.read_csv('dataset_sport-product.csv')
sdet=pd.read_csv('dataset_sport-detail.csv')

In [3]:
#Concatenating scraped tables
scr_cat = mcat.append(scat)
scr_prod=mprod.append(sprod)
scr_det=mdet.append(sdet)

## Clients data

In [4]:
#Setting data types and key columns parsing

new['Produktový kód'] = new['Produktový kód'].str.extract('(\d+)', expand=False)
new.dropna(subset=['Produktový kód'])
old.dropna(subset=['Kód výrobku'])
old=old.convert_dtypes()
old['Kód výrobku']=old['Kód výrobku'].astype(str)

#Merging catalogs to add missing product name for new table

prod=pd.merge(new,old, left_on=['Produktový kód'], right_on=['Kód výrobku'], how='left')
prod['Název produktu'] = np.where(prod['Název produktu'] == '-1', prod['Popis výrobku'], prod['Název produktu'])

In [5]:
#keep only items from recent years (>2017)

prod['Rok']=prod['Sezóna_x'].str.extract('(\d+)')
prod['Rok']=[i.lstrip('0') for i in prod['Rok'].astype(str)]
prod=prod[prod['Rok']!='nan']
prod['Rok']=prod['Rok'].astype(int)
prod=prod[prod['Rok']>17]


In [6]:
# Splitting category info into separate columns

cat_details = prod['Kategorie'].str.split('-', n=6, expand=True)
prod['Material']=cat_details[0]
prod['Sex']=cat_details[1]
prod['Type']=cat_details[2]
prod["Specification"]=cat_details[3]
prod["Other specification"]=cat_details[4]

## Scraped data

In [7]:
#remove leading letter in product_id column and drop duplicated product codes
scr_prod['product_id']=scr_prod['product_id'].str[1:]
scr_prod=scr_prod.drop_duplicates(subset=['product_id'], keep='last', inplace=False)

# Delete columns containing either 65% or more than 65% NaN Values 
perc = 65.0
min_count =  int(((100-perc)/100)*scr_det.shape[0] + 1)
scr_det = scr_det.dropna( axis=1, 
                thresh=min_count)

In [8]:
#convert data types
scr_det=scr_det.dropna(subset=['product_id'])
scr_det.loc[:, ['product_name']]=scr_det.loc[:, ['product_name']].fillna('-')
scr_det['product_id']=scr_det['product_id'].astype(str)

In [9]:
#Reshape data frame using values from parameter_0_name columns as column headers and parameter_0_label as values
scr_det_reshape=scr_det.pivot(index='product_id', columns='parameter/0/parameter_name', values='parameter/0/parameter_label').reset_index()
scr_det_reshape = scr_det_reshape.rename_axis(None, axis=1) 
scr_det_reshape = scr_det_reshape.rename_axis(None, axis=0)
scr_det_reshape=scr_det_reshape[['product_id', 'Výrobce', 'Výrobca', 'Značka']]
scr_det_reshape['Výrobce']=np.where(scr_det_reshape['Výrobce'].isna(), scr_det_reshape['Výrobca'], scr_det_reshape['Výrobce'])

In [10]:
#Reshape data frame using values from parameter_1_name columns as column headers and parameter_1_label as values
scr_det_reshape1=scr_det.pivot(index='product_id', columns='parameter/1/parameter_name', values='parameter/1/parameter_label').reset_index()
scr_det_reshape1 = scr_det_reshape1.rename_axis(None, axis=1) 
scr_det_reshape1 = scr_det_reshape1.rename_axis(None, axis=0)
scr_det_reshape1=scr_det_reshape1[['product_id', 'Výrobce', 'Značka']]

In [11]:
#Join reshaped tables
scr_det_resh=pd.merge(scr_det_reshape, scr_det_reshape1, on='product_id', how='inner')
scr_det_resh['Výrobce_x']=np.where(scr_det_resh['Výrobce_x'].isna(),scr_det_resh['Výrobce_y'], scr_det_resh['Výrobce_x'])
scr_det_resh['Značka_x']=np.where(scr_det_resh['Značka_x'].isna(),scr_det_resh['Značka_y'], scr_det_resh['Značka_x'])
scr_det_resh=scr_det_resh[['product_id', 'Výrobce_x','Značka_x']]
scr_det_resh=scr_det_resh.rename(columns={"Výrobce_x": "Výrobce", "Značka_x": "Značka"})
scr_det=scr_det[['product_id', 'product_name', 'product_url', 'category_name','category_tree']]
scr_det=pd.merge(scr_det, scr_det_resh, on='product_id', how='inner')

In [12]:
scr_cat=scr_cat.drop_duplicates(subset=['category_name'], keep='last', inplace=False)
scr_det=pd.merge(scr_det, scr_cat, on='category_name', how='left')
scr_det=scr_det.convert_dtypes()
scr_det['product_id']=scr_det['product_id'].astype(str)
scr_det['product_id']=scr_det['product_id'].str.replace('.0', '')
scr_det=scr_det.drop(['category_tree_y'], axis=1)
scr_det=scr_det.rename(columns={'category_tree_x':'category_tree'})

In [13]:
scr_prod=pd.merge(scr_prod, scr_det, on='product_id', how="left")
scr_prod=scr_prod.drop(['product_name_y','product_url_y'], axis=1)
scr_prod=scr_prod.rename(columns={'product_name_x':'product_name', 'product_url_x':'product_url'})

In [14]:
def parse_url(product_urls, re_string):
    '''For a given product urls find give re_string a returns list of matches.'''
    stor = []
    for url in product_urls:
        match = re.search(re_string, url)
        if match is not None:
            result = match.group(1)
            stor.append(result)
        else:
            stor.append('')
            
    return stor